In [1]:
import pandas as pd
import vk_api
import random
import numpy as np

In [2]:
ads = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ti6Su94955DZ4Tky8EbwifpgZf_dTjpBdiVH0Ukhsq94jZdqoHuUytZsFZKfwpXEUCKRFteJRc9P/pub?gid=889004448&single=true&output=csv')

In [3]:
ads

,date,time,event,platform,ad_id,ad_cost_type,ad_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,187.4
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,187.4
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
3,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,187.4
...,...,...,...,...,...,...,...
602,2019-04-02,2019-04-01 22:39:16,click,android,121288,CPM,187.4
603,2019-04-02,2019-04-01 22:40:29,click,android,121288,CPM,187.4
604,2019-04-02,2019-04-01 22:40:31,click,web,121288,CPM,187.4
605,2019-04-02,2019-04-01 22:44:14,click,ios,121288,CPM,187.4


In [4]:
# количество показов
# количество кликов
# CTR 
# сумма потраченных денег 


In [5]:
ads_views = ads[ads.event == 'view'].groupby(['date', 'ad_id']).count().reset_index()[['date', 'event']]
ads_views.columns = ['date', 'views']
ads_clicks = ads[ads.event == 'click'].groupby(['date', 'ad_id']).count().reset_index()[['date', 'event']]
ads_clicks.columns = ['date', 'clicks']

In [6]:
ads_views

,date,views
0,2019-04-01,491
1,2019-04-02,93


In [7]:
ads_clicks

,date,clicks
0,2019-04-01,17
1,2019-04-02,6


In [8]:
ads_ctr = ads_views.merge(ads_clicks, on='date')
ads_ctr = ads_ctr.assign(CTR = ads_ctr.clicks/ads_ctr.views)
ads_ctr.assign(CTR_PERCENT = 100 * ads_ctr.clicks/ads_ctr.views)

,date,views,clicks,CTR,CTR_PERCENT
0,2019-04-01,491,17,0.034623,3.462322
1,2019-04-02,93,6,0.064516,6.451613


In [9]:
ads['ad_action_cost'] = ads.ad_cost/1000
ads_ctr['money'] = ads['ad_action_cost'].unique()[0] * ads_ctr['views']

In [10]:
ads_ctr

,date,views,clicks,CTR,money
0,2019-04-01,491,17,0.034623,92.0134
1,2019-04-02,93,6,0.064516,17.4282


In [11]:
# Отчет по объявлению 121288 за 2 апреля
# Траты: Х рублей (Y%)
# Показы: X (Y%)
# Клики: X (-Y%)
# CTR: X (-Y%)

In [12]:
money_2_04 = float(ads_ctr[ads_ctr.date == '2019-04-02']['money'])
view_2_04 = float(ads_ctr[ads_ctr.date == '2019-04-02']['views'])
clicks_2_04 = float(ads_ctr[ads_ctr.date == '2019-04-02']['clicks'])
ctr_2_04 = float(ads_ctr[ads_ctr.date == '2019-04-02']['CTR'])

In [13]:
money_1_04 = float(ads_ctr[ads_ctr.date == '2019-04-01']['money'])
view_1_04 = float(ads_ctr[ads_ctr.date == '2019-04-01']['views'])
clicks_1_04 = float(ads_ctr[ads_ctr.date == '2019-04-01']['clicks'])
ctr_1_04 = float(ads_ctr[ads_ctr.date == '2019-04-01']['CTR'])

In [23]:
diff_money = round((money_2_04 - money_1_04)/money_1_04 * 100, 0)
diff_views = round((view_2_04 - view_1_04)/ view_1_04 * 100, 0)
diff_clicks = round((clicks_2_04 - clicks_1_04)/ clicks_1_04 * 100, 0)
diff_ctr = round((ctr_2_04 - ctr_1_04)/ctr_1_04*100, 0)

In [24]:
message_vk = f'''Отчет по объявлению 121288 за 2 апреля\n
# Траты: {money_2_04} рублей ({diff_money}%)
# Показы: {view_2_04} ({diff_views}%)
# Клики: {clicks_2_04} ({diff_clicks}%)
# CTR: {ctr_2_04} ({diff_ctr})%
'''

In [26]:
message_vk

'Отчет по объявлению 121288 за 2 апреля\n\n# Траты: 17.4282 рублей (-81.0%)\n# Показы: 93.0 (-81.0%)\n# Клики: 6.0 (-65.0%)\n# CTR: 0.06451612903225806 (86.0)%\n'

In [16]:
with open(f'report_2_04.txt', 'w') as file:
    file.write(message_vk)

In [17]:
app_token = '4e5a1c298f4c98468654d34b48cdcc03647593500f78a9529e463582e0c71f7f0acd03725b18aa2977267'
chat_id = 1
my_id = 533514830

vk_session = vk_api.VkApi(token=app_token)
vk = vk_session.get_api()

vk.messages.send(
    chat_id=chat_id,
    random_id=random.randint(1, 2 ** 31),
    message=message_vk)



0

In [18]:
ads.value_counts('event')

event
view     584
click     23
dtype: int64